<a href="https://colab.research.google.com/github/dubisx/EMCT_final/blob/main/Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
!pip install miditok
!pip install tokenizers
!pip install transformers

import tensorflow as tf
from tensorflow import keras
import numpy
import miditok
import tqdm
from miditok import MIDILike, MIDITokenizer, REMI
import pathlib
from miditok.constants import CHORD_MAPS
from transformers import TFGPT2LMHeadModel, GPT2Config, Trainer, TrainingArguments, GenerationConfig, pipeline

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [24]:
from google.colab import drive

drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [29]:
model_path = "/content/gdrive/MyDrive/model_f/model5/"

pitch_range = range(21, 109)
beat_res = {(0, 4): 8, (4, 12): 4}
nb_velocities = 32
additional_tokens = {'Chord': True, 'Rest': True, 'Tempo': True,
                     'rest_range': (2, 8),  # (half, 8 beats)
                     'nb_tempos': 32,  # nb of tempo bins
                     'tempo_range': (40, 250),  # (min, max)
                     'Program': False,
                     "chord_maps": CHORD_MAPS,
                     "chord_tokens_with_root_note": True,
                     "chord_unknown": False}
special_tokens = ["PAD", "BOS", "EOS"]

tokenizer = MIDILike(pitch_range, beat_res, nb_velocities, additional_tokens, special_tokens=special_tokens)

print('Loading the model...')

# Creates model
config = GPT2Config(
    vocab_size=1000,
    n_positions=10,
    n_embd=512,
    n_layer=8,
    n_head=8,
    n_inner=2048,
    resid_pdrop=.1,
    embd_pdrop=.1,
    attn_pdrop=.1,
    padding_token_id=tokenizer['PAD_None'],
    bos_token_id=tokenizer['BOS_None'],
    eos_token_id=tokenizer['EOS_None'],
)
model = TFGPT2LMHeadModel(config)

Loading the model...


In [30]:
model.from_pretrained(model_path)

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at /content/gdrive/MyDrive/model_f/model5/.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [31]:

# defining our optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0)
# definining our loss function
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
# defining our metric which we want to observe
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
# compiling the model
model.compile(optimizer=optimizer, loss=[loss, *[None] * model.config.n_layer], metrics=[metric])

In [32]:
from pathlib import Path
from miditok.utils import get_midi_programs
from miditoolkit import MidiFile


midi_path = '/content/gdrive/MyDrive/example midi/midi_example.mid'

tokenizer.learn_bpe(
    vocab_size=1900,
    tokens_paths=list(Path("/content/gdrive/MyDrive/tokens_noBPE").glob('**/*.json')),
    out_dir=Path('/content/gdrive/MyDrive/tokens_BPE')
)

Loading token files: 100%|██████████| 5456/5456 [00:08<00:00, 622.44it/s] 


In [33]:
midi_path = '/content/gdrive/MyDrive/example midi/midi/midi_example.mid'

midi = MidiFile(midi_path)
token = tokenizer(midi)  # automatically detects MidiFile, paths or tokens before converting them


In [34]:
print(token[0].ids)

[328, 904, 48, 204, 52, 438, 21, 201, 226, 109, 24, 461, 112, 17, 1139, 702, 1106, 271, 38, 204, 1627, 45, 438, 64, 205, 217, 958, 64, 201, 65, 347, 152, 213, 64, 1139, 882, 105, 16, 201, 217, 587, 211, 126, 104, 679, 794, 45, 204, 1075, 222, 1284, 67, 534, 1160, 155, 64, 862, 1106, 882, 102, 129, 520, 1893, 1014, 389, 226, 128, 40, 204, 62, 202, 217, 958, 740, 201, 217, 131, 135, 958, 104, 128, 904, 48, 204, 52, 438, 21, 201, 226, 109, 24, 461, 112, 17, 1139, 136, 702, 133, 271, 38, 204, 1627, 904, 234, 105, 16, 201, 217, 1318, 211, 104, 679, 794, 45, 204, 1075, 241, 587, 820, 102, 520, 1893, 1014, 389, 226, 128, 40, 204, 225, 131, 135, 780, 104]


In [35]:
# savetok = Path('/content/gdrive/MyDrive/example midi/token1.json')

# tokenizer.save_tokens(token, savetok)

In [36]:
import json
import os

directory = '/content/gdrive/MyDrive/example midi/tokens/'


for fname in os.listdir(directory):                     # for each file in the directory
    with open(os.path.join(directory, fname)) as i:     # open the file
        data = json.load(i)   

In [37]:
# print(data)

In [38]:
prompt = [data['ids'][0][:300]]


 I have managed to convert a new MIDI to a BPE encoded token, however I am unsure on how to feed the data into the model.generate function to generate a new sequence.

In [39]:
generation_config = GenerationConfig(
    max_new_tokens=512,  # extends samples by 512 tokens
    num_beams=1,        # no beam search
    do_sample=True,     # but sample instead
    temperature=0.9,
    top_k=15,
    top_p=0.95,
    epsilon_cutoff=3e-4,
    eta_cutoff=1e-3,
    pad_token_id=config.padding_token_id,
)

In [40]:
outputs = model.generate(prompt, generation_config)

In [41]:
outputs = tf.Variable(outputs).numpy().tolist()
print('old token:', token[0].ids)

new_data = token
new_data[0].ids = outputs

print('new token:', new_data[0].ids)

old token: [328, 904, 48, 204, 52, 438, 21, 201, 226, 109, 24, 461, 112, 17, 1139, 702, 1106, 271, 38, 204, 1627, 45, 438, 64, 205, 217, 958, 64, 201, 65, 347, 152, 213, 64, 1139, 882, 105, 16, 201, 217, 587, 211, 126, 104, 679, 794, 45, 204, 1075, 222, 1284, 67, 534, 1160, 155, 64, 862, 1106, 882, 102, 129, 520, 1893, 1014, 389, 226, 128, 40, 204, 62, 202, 217, 958, 740, 201, 217, 131, 135, 958, 104, 128, 904, 48, 204, 52, 438, 21, 201, 226, 109, 24, 461, 112, 17, 1139, 136, 702, 133, 271, 38, 204, 1627, 904, 234, 105, 16, 201, 217, 1318, 211, 104, 679, 794, 45, 204, 1075, 241, 587, 820, 102, 520, 1893, 1014, 389, 226, 128, 40, 204, 225, 131, 135, 780, 104]
new token: [[328, 904, 48, 204, 52, 438, 21, 201, 226, 109, 24, 461, 112, 17, 204, 213, 702, 133, 136, 271, 38, 204, 41, 201, 45, 438, 64, 205, 217, 958, 64, 201, 65, 347, 152, 213, 64, 204, 213, 882, 105, 16, 201, 217, 587, 211, 126, 104, 679, 794, 45, 204, 48, 202, 222, 152, 214, 67, 534, 65, 338, 155, 64, 862, 133, 136, 882, 102

Not sure what the issue is, as the data seems to be in the correct type.



Tried changing architecture, as suggested, however now I am getting this error.


In [42]:
generated_midi1 = tokenizer.tokens_to_midi(new_data[0].ids)

In [ ]:
programs = get_midi_programs(midi)

In [ ]:
generated_midi = tokenizer(new_data[0].ids, programs=programs)

KeyError: ignored

In [43]:
generated_midi1.dump('/content/gdrive/MyDrive/example midi/file.mid')